In [1]:
model_prefix="Cameron/BERT-"
models = [model_prefix + x for x in ["Jigsaw", "SBIC-offensive", "SBIC-targetcategory", "eec-emotion", "jigsaw-identityhate", "jigsaw-severetoxic", "mdgender-convai-binary", "mdgender-convai-ternary", "mdgender-wizard", "rtgender-opgender-annotations"]]

In [2]:
models

['Cameron/BERT-Jigsaw',
 'Cameron/BERT-SBIC-offensive',
 'Cameron/BERT-SBIC-targetcategory',
 'Cameron/BERT-eec-emotion',
 'Cameron/BERT-jigsaw-identityhate',
 'Cameron/BERT-jigsaw-severetoxic',
 'Cameron/BERT-mdgender-convai-binary',
 'Cameron/BERT-mdgender-convai-ternary',
 'Cameron/BERT-mdgender-wizard',
 'Cameron/BERT-rtgender-opgender-annotations']

In [3]:
model_relabel = {
    'Cameron/BERT-Jigsaw': [], # unknown
    'Cameron/BERT-SBIC-offensive': ["Not", "Maybe", "Offensive"],
    'Cameron/BERT-SBIC-targetcategory': [], # incorrect
    'Cameron/BERT-eec-emotion': ['none', 'anger','fear', 'joy', 'sadness'],
    'Cameron/BERT-jigsaw-identityhate': ['Not', "Yes"],
    'Cameron/BERT-jigsaw-severetoxic': ["Not", "Yes"],
    'Cameron/BERT-mdgender-convai-binary': ["female", "male"],
    'Cameron/BERT-mdgender-convai-ternary': ["female", "male", "neutral"],
    'Cameron/BERT-mdgender-wizard': ["neutral", "female", "male"],
    'Cameron/BERT-rtgender-opgender-annotations': [] # incorrect
}

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [5]:
pretrained_models = {}
for model in models:
    print(f"Loading {model}")
    pretrained_models[model] = AutoModelForSequenceClassification.from_pretrained(model)

Loading Cameron/BERT-Jigsaw
Loading Cameron/BERT-SBIC-offensive
Loading Cameron/BERT-SBIC-targetcategory
Loading Cameron/BERT-eec-emotion
Loading Cameron/BERT-jigsaw-identityhate
Loading Cameron/BERT-jigsaw-severetoxic
Loading Cameron/BERT-mdgender-convai-binary
Loading Cameron/BERT-mdgender-convai-ternary
Loading Cameron/BERT-mdgender-wizard
Loading Cameron/BERT-rtgender-opgender-annotations


In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [7]:
# find all the models with labels that we've defined
models_with_labels = []

for model_name, val in model_relabel.items():
    if len(val):
        models_with_labels.append(model_name)

In [8]:
models_with_labels

['Cameron/BERT-SBIC-offensive',
 'Cameron/BERT-eec-emotion',
 'Cameron/BERT-jigsaw-identityhate',
 'Cameron/BERT-jigsaw-severetoxic',
 'Cameron/BERT-mdgender-convai-binary',
 'Cameron/BERT-mdgender-convai-ternary',
 'Cameron/BERT-mdgender-wizard']

In [13]:
def predict(sentence, model_name):
    encoded_sentence = tokenizer.encode_plus(SENTENCE, return_tensors="pt")
    sentence_logits = pretrained_models[model_name](**encoded_sentence)[0]
    #print(sentence_logits)
    softmaxes = torch.softmax(sentence_logits, dim=1).tolist()[0]
    labels = model_relabel[model_name]
    prediction_str = {}
    for label, softmax in zip(labels, softmaxes):
        prediction_str[label] = softmax * 100
    print(f"{model_name}: {prediction_str}")
    return prediction_str

In [14]:
SENTENCE = """
THIS IS A SUPER TOXIC SENTENCE WE ARE TESTING
"""

In [15]:
preds = {}
for model_name in models_with_labels:
    preds[model_name] = predict(SENTENCE, model_name)


Cameron/BERT-SBIC-offensive: {'Not': 99.76773858070374, 'Maybe': 0.14955037040635943, 'Offensive': 0.08271938422694802}
Cameron/BERT-eec-emotion: {'none': 4.477528482675552, 'anger': 0.04400764009915292, 'fear': 3.006780706346035, 'joy': 92.41065979003906, 'sadness': 0.06103508058004081}
Cameron/BERT-jigsaw-identityhate: {'Not': 99.9485969543457, 'Yes': 0.051401183009147644}
Cameron/BERT-jigsaw-severetoxic: {'Not': 99.968421459198, 'Yes': 0.0315822078846395}
Cameron/BERT-mdgender-convai-binary: {'female': 0.4176512826234102, 'male': 99.58235025405884}
Cameron/BERT-mdgender-convai-ternary: {'female': 0.026768672978505492, 'male': 0.20704201888293028, 'neutral': 99.76619482040405}
Cameron/BERT-mdgender-wizard: {'neutral': 1.3239141553640366, 'female': 1.4711951836943626, 'male': 97.20488786697388}
